Testing of the method


In [ ]:
#Doesent work download manually instead

# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text),"\n")
print(text[:1000])

chars = sorted(list(set(text)))
vocab_size = len(chars)
print('All characters in dataset')
print(''.join(chars))
print('There are ',vocab_size,' unique characters')

length of dataset in characters:  1115393 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hu

In [ ]:
stoi = {ch:i for i,ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]          #Endcoder: string --> list of integers
decode = lambda l: ''.join([itos[i] for i in l])  #Decoder: list of integers --> string

testString = "Test string & chars"
print(encode(testString))
print(decode(encode(testString)))

[32, 43, 57, 58, 1, 57, 58, 56, 47, 52, 45, 1, 4, 1, 41, 46, 39, 56, 57]
Test string & chars


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) -block_size, (batch_size,)) #random offset in the dataset from 0 to len(data)-block_size
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
    print('Batch: ', b)
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1, 39, 45, 39, 47, 52, 57, 58],
        [26, 53,  1, 51, 39, 58, 58, 43],
        [ 1, 61, 43, 43, 54, 47, 52, 45],
        [63,  6,  7,  7,  0,  0, 13, 33]])
targets
torch.Size([4, 8])
tensor([[39, 45, 39, 47, 52, 57, 58,  1],
        [53,  1, 51, 39, 58, 58, 43, 56],
        [61, 43, 43, 54, 47, 52, 45,  1],
        [ 6,  7,  7,  0,  0, 13, 33, 18]])
-----
Batch:  0
when input is [1] the target: 39
when input is [1, 39] the target: 45
when input is [1, 39, 45] the target: 39
when input is [1, 39, 45, 39] the target: 47
when input is [1, 39, 45, 39, 47] the target: 52
when input is [1, 39, 45, 39, 47, 52] the target: 57
when input is [1, 39, 45, 39, 47, 52, 57] the target: 58
when input is [1, 39, 45, 39, 47, 52, 57, 58] the target: 1
Batch:  1
when input is [26] the target: 53
when input is [26, 53] the target: 1
when input is [26, 53, 1] the target: 51
when input is [26, 53, 1, 51] the target: 39
when input is [26, 53, 1, 51, 39] the target: 58
w

Full finished code

Start here:

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text),"\n")
print(text[:1000])

chars = sorted(list(set(text)))
vocab_size = len(chars)
print('All characters in dataset')
print(''.join(chars))
print('There are ',vocab_size,' unique characters')

length of dataset in characters:  1115393 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hu

In [ ]:
stoi = {ch:i for i,ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]          #Endcoder: string --> list of integers
decode = lambda l: ''.join([itos[i] for i in l])  #Decoder: list of integers --> string

testString = "Test string & chars"
print(encode(testString))
print(decode(encode(testString)))

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

[32, 43, 57, 58, 1, 57, 58, 56, 47, 52, 45, 1, 4, 1, 41, 46, 39, 56, 57]
Test string & chars
torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  

In [ ]:
#Hyperparameters Currently to big for google colab
batch_size = 64       #was 32
block_size = 128        #was 8
max_iters = 5000      #was 5000
eval_interval = 200   #was 300
learning_rate = 3e-4  #was 1e-3
eval_iters = 200      #was 200
n_embd = 256           #was 32
n_head = 6
n_layer = 6
dropout = 0.2


In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) -block_size, (batch_size,)) #random offset in the dataset from 0 to len(data)-block_size
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [ ]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout) #Randomly shuts of some neurons to prevent overfitting

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)   #Randomly shuts of some neurons to prevent overfitting

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
          nn.Linear(n_embd,4 * n_embd),
          nn.ReLU(),
          nn.Linear(4 * n_embd, n_embd),
          nn.Dropout(dropout),   #Randomly shuts of some neurons to prevent overfitting
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):

    def __init__(self,n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))    #Residual connections is why we keep x
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (Batch,Time) tensor of integers
        tok_emb = self.token_embedding_table(idx) #(Batch, Time, Chanel) 3-dimensionall tensor
        pos_emb = self.position_embedding_table(torch.arange(T))  # (T,C)
        x = tok_emb + pos_emb   #(B,T,C)
        x = self.blocks(x)       #(B,T,C)
        logits =  self.lm_head(x) #(B,T,voacb_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits=logits.view(B*T, C) #Fits better with pytorch
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss


    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

xb,yb = get_batch('train')

m = BigramLanguageModel()
logits, loss = m(xb,yb)
print(loss)

#idx = torch.zeros((1,1), dtype=torch.long)
#print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

tensor(4.5779, grad_fn=<NllLossBackward0>)


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step  {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



step  0: train loss 4.5740, val loss 4.5672
step  200: train loss 2.4228, val loss 2.4485
step  400: train loss 2.1800, val loss 2.2149
step  600: train loss 1.9958, val loss 2.0626
step  800: train loss 1.8524, val loss 1.9601
step  1000: train loss 1.7507, val loss 1.8874
step  1200: train loss 1.6736, val loss 1.8306
step  1400: train loss 1.6134, val loss 1.7824
step  1600: train loss 1.5693, val loss 1.7486
step  1800: train loss 1.5273, val loss 1.7158
step  2000: train loss 1.5001, val loss 1.6967
step  2200: train loss 1.4691, val loss 1.6642
step  2400: train loss 1.4443, val loss 1.6507
step  2600: train loss 1.4228, val loss 1.6271
step  2800: train loss 1.4054, val loss 1.6171
step  3000: train loss 1.3896, val loss 1.5939
step  3200: train loss 1.3730, val loss 1.5903
step  3400: train loss 1.3587, val loss 1.5727
step  3600: train loss 1.3478, val loss 1.5778
step  3800: train loss 1.3339, val loss 1.5657
step  4000: train loss 1.3235, val loss 1.5519
step  4200: train lo

In [ ]:
#Generate from model
context = torch.zeros((1,1), dtype=torch.long)

print(context)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


tensor([[0]])


MENENIUS:
Mercy, one dreams; why, counsel the hopesition.

CORIOLANUS:
Yet be bankle, thy heaven worse,
Your twos are me, you comed's tongue rosen:
Hosteff, each your soul means with our poorsue.

DORSET:
Not should usband and I hard, with to Poluck this Engling;
My lies: this over are not, for our serve
My honest at his name of ponior tongue,
Some remied with some and from me and sooth,
I come him to give him.

HENRY Provingman:
The consman's soul, what honour thank his good were,
To other sol
